In [1]:
import os

In [2]:
import sys
print(sys.executable)


C:\Users\det5cob\.conda\envs\chromadbexp\python.exe


In [3]:
# Set the go to File > Preferences > Settings > User tab > Extensions > python > Python Path  -- Setting.json 

In [3]:
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

### Clone Github repositories

In [4]:
%pwd

'C:\\Backup_old_laptop\\Machine Learning\\Genrative_AI\\1iNeuron_Course\\20_30APR_24_\\CODE_ANALYSIS_USING_GENAI\\research'

In [34]:
!mkdir test_repo

In [35]:
repo_path = "test_repo/"

Repo.clone_from("https://github.com/thangarajdeivasikamani/MCQ_OPENAPI", to_path=repo_path)

<git.repo.base.Repo 'C:\\Backup_old_laptop\\Machine Learning\\Genrative_AI\\1iNeuron_Course\\20_30APR_24_\\CODE_ANALYSIS_USING_GENAI\\research\\test_repo\\.git'>

In [36]:
repo_path = "test_repo/"

loader = GenericLoader.from_filesystem(repo_path+'/',
                                        glob = "**/*",
                                       suffixes=[".py"],
                                       parser = LanguageParser(language=Language.PYTHON, parser_threshold=500)
)

In [37]:
documents = loader.load()

In [38]:
documents

[Document(page_content='from setuptools import find_packages,setup\n\nsetup(\n    name=\'mcqgenrator\',\n    version=\'0.0.1\',\n    author=\'Thangaraj\',\n    author_email=\'thangarajerode@gmail.com\',\n    install_requires=["openai","langchain","streamlit","python-dotenv","PyPDF2"],\n    packages=find_packages()\n)', metadata={'source': 'test_repo\\setup.py', 'language': <Language.PYTHON: 'python'>}),
 Document(page_content='import os\nimport json\nimport traceback\nimport pandas as pd\nfrom dotenv import load_dotenv\nfrom mcqgenrator.utils import read_file,get_table_data\nimport streamlit as st\nfrom langchain.callbacks import get_openai_callback\nfrom mcqgenrator.MCQGenrator import generate_evaluate_chain\nfrom mcqgenrator.logger import logging\n\n#loading json file\n\nwith open(\'Response.json\', \'r\') as file:\n    RESPONSE_JSON = json.load(file)\n\n#creating a title for the app\nst.title("MCQs Creator Application with LangChain 🦜⛓️")\n\n#Create a form using st.form\nwith st.for

### Chunkings

In [39]:
documents_splitter = RecursiveCharacterTextSplitter.from_language(language = Language.PYTHON,
                                                             chunk_size = 2000,
                                                             chunk_overlap = 200)

In [40]:
texts = documents_splitter.split_documents(documents)

In [41]:
len(texts)

8

### Embedding model

In [42]:
os.environ["OPENAI_API_KEY"] = "************************"

In [43]:
embeddings=OpenAIEmbeddings(disallowed_special=())

### Knowledge base (vector DB)

In [44]:
vectordb = Chroma.from_documents(texts, embedding=embeddings, persist_directory='./data')
vectordb.persist()

### LLM Wrapper

In [45]:
# llm = ChatOpenAI(model_name="gpt-4")
llm = ChatOpenAI()

C:\Users\det5cob\.conda\envs\chromadbexp\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [46]:
memory = ConversationSummaryMemory(llm=llm, memory_key = "chat_history", return_messages=True)

In [47]:
qa = ConversationalRetrievalChain.from_llm(llm, retriever=vectordb.as_retriever(search_type="mmr", search_kwargs={"k":3}), memory=memory)

### Q&A

In [48]:
question = "what i mean by st.spinner?"

In [49]:
result = qa(question)
print(result['answer'])

C:\Users\det5cob\.conda\envs\chromadbexp\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Number of requested results 20 is greater than number of elements in index 8, updating n_results = 8


`st.spinner()` is a function in Streamlit, a popular Python library used for creating web applications with simple interactive elements. When you use `st.spinner()`, it displays a spinner animation on the web application interface, indicating to the user that some process is ongoing, such as loading data or calculations being performed. It is a visual cue to inform the user that the application is working on a task and they should wait for the process to complete.
